# Små Øvelser – LOG decorator

In [ ]:
from __future__ import annotations
import time
import datetime as dt
from functools import wraps

In [22]:
import datetime

def log_decorator(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        with open("log.txt", "a") as f:
            f.write(
                f"[{datetime.datetime.now()}] "
                f"Function {func.__name__} called with args={args}, kwargs={kwargs}, "
                f"result={result}\n"
            )
        return result
    return wrapper


In [23]:
@log_decorator
def add(*args):
    return sum(args)

add(1, 2, 3)
add(10, 5)


15

In [24]:
@log_decorator
def printer(text):
    return text

print(printer("Hej verden!"))


Hej verden!


# Ex1: Time it! – timing decorator

In [30]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        try:
            return func(*args, **kwargs)
        finally:
            end = time.perf_counter()
            print(f"{func.__name__} took {end - start:.6f} s")
    return wrapper

In [31]:
@timeit
def heavy_work(n: int) -> int:
    return sum(range(n))

heavy_work(1_000_000)

heavy_work took 0.015546 s


499999500000

# Ex3: Slow down code – slowdown decorator

In [27]:
def slowdown(seconds: float = 1.0):
    """Dekoratørfabrik: Sænker hastighed med 'seconds' før HVERT funktionskald."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            time.sleep(seconds)
            return func(*args, **kwargs)
        return wrapper
    return decorator


@slowdown(1.0)  # ét sekund mellem hvert rekursive kald
def countdown_recursive(n: int):
    if not n:  # 0 er falsy
        print("Liftoff!")
        return n
    else:
        print(n, end=" ")
        return countdown_recursive(n - 1)


def countdown_loop(n: int):
    # Normal løkke-version uden decorator (viser kontrast)
    while n > 0:
        print(n, end=" ")
        n -= 1
        # Hvis du vil have 1s pause pr. trin i løkkeversionen,
        # så sov her, ikke i dekorationen:
        time.sleep(1)
    print("Liftoff!")


# BONUS: Hvis du *insisterer* på at bruge decorator til løkkeversionen,
# kan du pakke printet i en hjælperfunktion og dekorere den:
@slowdown(1.0)
def _tick_print(x):
    print(x, end=" ")

def countdown_loop_with_decorator(n: int):
    while n > 0:
        _tick_print(n)  # bliver forsinket via decorator pr. iteration
        n -= 1
    print("Liftoff!")

# Ex4: Decorating Game Characters – evne/skill decorators

In [28]:
def skill(name: str):
    """Generisk dekoratør der tilføjer en færdighed til den returnerede beskrivelse."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            base = func(*args, **kwargs)
            # Tilføj teksten på en pæn måde
            # Hvis base allerede indeholder komma/og-liste, hæft på med komma.
            if "I'm" in base and "character" in base:
                # Ensartet slutning: ", the <skill>" eller ", and <skill>" if already one appended
                separator = ", "
                return f"{base}{separator}{name}"
            else:
                return f"{base}, {name}"
        return wrapper
    return decorator

# Konkrete skills bygges fra skill()
sharpshooter = skill("the sharpshooter character")
stealthy     = skill("the stealthy character")
hacker       = skill("the hacker character")

@sharpshooter
@stealthy
def player():
    return "I'm the player character"


In [34]:
from pathlib import Path

LOG_PATH = Path("log.txt")
if __name__ == "__main__":
    # Små Øvelser – log
    print("add(1,2,3) ->", add(1, 2, 3))
    print("printer('Hej!') ->", printer("Hej!"))
    print(f"Se log i {LOG_PATH.resolve()}")

    # Ex1 – timeit
    heavy_work(2_000_000)

    # Ex3 – slowdown: rekursiv
    print("\nCountdown (rekursiv, 1s pr. trin):")
    countdown_recursive(5)

    # Ex3 – slowdown: løkke m. direkte sleep
    print("\nCountdown (løkke, 1s pr. trin):")
    countdown_loop(5)

    # Ex3 – slowdown: løkke via dekorering af "tick"
    print("\nCountdown (løkke, 1s pr. trin via dekoreret tick):")
    countdown_loop_with_decorator(5)

    # Ex4 – stacking decorators
    print("\nGame character:")
    print(player())          # forventet: "I'm the player character, the stealthy character, the sharpshooter character"

    # Bonus – endnu flere evner:
    @hacker
    @sharpshooter
    @stealthy
    def elite():
        return "I'm the player character"
    print(elite())

add(1,2,3) -> 6
printer('Hej!') -> Hej!
Se log i C:\Users\adria\Desktop\Skole\4. Semester\Python\Python Gits\fall2025\materialer\decorators\log.txt
heavy_work took 0.040206 s

Countdown (rekursiv, 1s pr. trin):
5 4 3 2 1 Liftoff!

Countdown (løkke, 1s pr. trin):
5 4 3 2 1 Liftoff!

Countdown (løkke, 1s pr. trin via dekoreret tick):
5 4 3 2 1 Liftoff!

Game character:
I'm the player character, the stealthy character, the sharpshooter character
I'm the player character, the stealthy character, the sharpshooter character, the hacker character
